## Assignment 4, PROBLEM 1

This time the assignment only consists of one problem, but we will do a more comprehensive analysis instead.

Consider the dataset `mammography.mat` that you can download from [http://odds.cs.stonybrook.edu/mammography-dataset/](http://odds.cs.stonybrook.edu/mammography-dataset/). Below you can find the code to load this file into `X` and `Y`, you just need to put the file in your `data` folder. During mammography the doctor would be looking for something called `calcification`, which is calcium deposits in the breast tissue and is used as an indicator of cancer. In this dataset the `X` corresponds to some measurements, there are 6 features. The `Y` is a 0-1 label where 1 represents calcification and 0 does not.

### 1)
Split the data into three parts, train/test/validation where train is 60% of the data, test is 15% and validation is 25% of the data. Do not do this randomly, I have prepared a shuffling with a fixed seed, this way I can make sure that we all did the same splits. That is [train,test,validation] is the splitting layout.

In [1]:
import scipy.io as so
import numpy as np
data = so.loadmat('data/mammography.mat')

np.random.seed(0)
shuffle_index = np.arange(0,len(data['X']))
np.random.shuffle(shuffle_index)

X = data['X'][shuffle_index,:]
Y = data['y'][shuffle_index,:].flatten()

### Answer

Pretty straight forward, just regular indexing

In [2]:
# Split the X,Y into three parts, make sure that the sizes are
# (6709, 6), (1677, 6), (2797, 6), (6709,), (1677,), (2797,)
length = len(X)
X_train, X_test, X_valid, Y_train, Y_test, Y_valid = X[0:6709], X[6709:8386], X[8386:], Y[0:6709], Y[6709:8386], Y[8386:]

### 2)
Train a machine learning model on the training data (you are free to choose it yourself). Hint: you could always try `LogisticRegression`, but for it to work well you would need `StandardScaler` and put everything in a `Pipeline`.

In [3]:
# Part 2
# Use X_train,Y_train to train a model from sklearn. Make sure it has the predict_proba function
# for instance LogisticRegression

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

p = Pipeline([('scaler', StandardScaler()), ('model',LogisticRegression())])

model = p.fit(X_train, Y_train)

### 3)
Use the classification report from `Utils` and compute the intervals for precision-recall etc on the test set with `union_bound correction` with 95% confidence.

### Answer

Use the classification report interval from utils, it will display all the necessary information

In [4]:
# Compute precision and recall on the test set using 
from Utils import classification_report_interval

# Each precision and recall should be a tuple, for instance you can write
# precision0 = (0.9,0.95)

y_pred = p.predict(X_test)
print(classification_report_interval(Y_test, y_pred, alpha=0.05))

precision0 = (0.94,1.00)
recall0 = (0.96, 1.00)
precision1 = (0.33,1.00)
recall1 = (0.09,0.62)

            labels           precision             recall

                 0  0.98 : [0.94,1.00] 1.00 : [0.96,1.00]
                 1  0.70 : [0.33,1.00] 0.36 : [0.09,0.62]

          accuracy                                        0.98 : [0.94,1.00]



### 4)
You are interested in minimizing the average cost of your classifier, but first we must define it:
    * If someone is calcified but classified as not, we say it costs 30 **(this is the worst scenario)** 
    * If someone is not calcified but classified as calcified, we say it costs 5 **(this probably only costs extra investigation)**
    * If someone is calcified but classified as calcified, costs 0 **(We did the right thing, no cost)**
    * If someone is not calcified but classified as not, costs 0 **(We did the right thing, no cost)**.

complete filling the function `cost` to compute the cost of a prediction model under a certain prediction threshold (recall our precision recall lecture and the `predict_proba` function from trained models). What would be the cost of having a model that always classifies someone as not calcified on the test set?

### Answer

Let false negatives be denoted by $E_1$. We can model false negatives as

$$E_1 = 1_{Y=1, g(X)=0} = 1_{Y=1} 1_{g(X) = 0}$$

Where $Y$ is the outcome, and $g(X)$ is our prediction.

Let false positives be denoted by $E_2$. We get

$$E_2 = 1_{Y=0, g(X)=1} = 1_{Y=0} 1_{g(X) = 1}$$

In [5]:
def cost(model,threshold,X,Y):
    pred_proba = model.predict_proba(X)[:,1]
    predictions = (pred_proba >= threshold)*1
    
    # Fill in what is missing to compute the cost and return it
    # Note that we are interested in average cost (cost per person)
    
    Yp = predictions
    
    E_1 = Y * (1-Yp) # E1 is a vector that contains a 1 for every false negative, 0 everywhere else
    E_2 = (1-Y) * Yp # Same concept for E_2, but for false positives
    
    return np.mean(E_1*30 + E_2*5) # Weight E_1 and E_2 with costs, return mean


# Fill in the naive cost of the model that would classify all as non-calcified on the test set
Y_naive = np.zeros(len(Y_test))
naive_cost = np.mean(Y_test*(1-Y_naive)*30)

### 5)

Now, we wish to select the threshold of our classifier that minimizes the cost, we do that by checking say 100 evenly spaced proposal thresholds between 0 and 1, and use our testing data to compute the cost.

### Answer
Generate and loop through 100 evenly spaced thresholds $\in (0,1)$ and save the minimum.

In [6]:
# Part 5
minThresh = 50000
minCost = 50000
for i in np.linspace(0,1,100):
    iCost = cost(model,i,X_test,Y_test)
    if (iCost < minCost):
        minThresh = i
        minCost = iCost
        
optimal_threshold = minThresh
cost_at_optimal_threshold = minCost

### 6)

With your newly computed threshold value, compute the cost of putting this model in production by computing the cost using the validation data. Also provide a confidence interval of the cost using Hoeffdings inequality with a 95% confidence.

### Answer
We know that the upper bound for the cost function is $30$, that is, only false negatives. We calculate the epsilon with parameters 

$$n=|\text{validation set}|, b=30, \alpha = 0.05$$ 

We then use the confidence intervals function from utils to obtain the answer.

In [7]:
cost_at_optimal_threshold_validation = cost(model,minThresh,X_valid,Y_valid)
# Report the cost interval as a tuple cost_interval = (a,b)

from Utils import print_confidence_interval, epsilon_bounded, bennett_epsilon

eps = epsilon_bounded(len(Y_valid), 30, 0.05)
print_confidence_interval(cost_at_optimal_threshold_validation,eps,min_value=0,max_value=30)

cost_interval = (0.00, 1.06)

[0.00,1.06]


### 7)

Let $t$ be the threshold you found and $f$ the model you fitted, if we define the random variable
$$
    C = 30(1-1_{f(X)\geq t})Y+5(1-Y)1_{f(X) \geq t}
$$
then $C$ denotes the cost of a randomly chosen patient. In the above we are estimating $\mathbb{E}[C]$ using the empirical mean. However, since the number of calcifications in the population is fairly small and our classifier probably has a fairly high precision for the $0$ class, then $C$ should have fairly small variance. Compute the empirical variance of $C$ on the validation set. What would be the confidence interval if we used Bennett's inequality instead of Hoeffding in point 6 but with the computed empirical variance as our guess for the variance?

### Answer

Recall the formula for sample variance

$$
s^2 = \frac{\sum (X-\overline{X})^2}{n-1}
$$

We redefine the cost function to obtain the sample variance, then use the `bennet_epsilon`  function with

$$
n = |\text{validation set}|, b=30, \sigma = \sqrt{s^2}, \alpha = 0.05
$$

In [8]:
def costVariance(model,threshold,X,Y):
    pred_proba = model.predict_proba(X)[:,1]
    predictions = (pred_proba >= threshold)*1
    
    Yp = predictions
    
    E_1 = Y * (1-Yp)
    E_2 = (1-Y) * Yp
    
    costs = E_1*30 + E_2*5
    mean = np.mean(costs)
    varSum = np.sum(np.power(costs-mean,2))
    nMin1 = len(Y)-1
    
    return varSum/nMin1

variance_of_C = costVariance(model,optimal_threshold,X_valid,Y_valid)

# A 95% confidence interval of the random variable C using Bennett's inequality
bennyEps = bennett_epsilon(len(Y_valid), 30,np.sqrt(variance_of_C), 0.05)
print_confidence_interval(cost_at_optimal_threshold_validation,bennyEps,min_value=0,max_value=30)
interval_of_C = (0.15, 0.43)

Numerical error 1.0408340855860843e-17
[0.15,0.43]
